In [111]:
reTrain = False
reTrainHyper = False
reFetch = False

stats_url = 'https://stats.csconfederation.com/graphql'


In [112]:
import requests
import pandas as pd
from fastai.tabular.all import *
from fastai.data.transforms import RandomSplitter
from fastai.metrics import *
tier_dict = {'Recruit': 1, 'Prospect': 2, 'Contender': 3, 'Challenger': 4, 'Elite': 5, 'Premier': 6}
active_seasons = [13, 14, 15, 16]


In [113]:
import torch
print(torch.cuda.is_available())


True


In [114]:
def feature_scale(series):
    max = series.max()
    min = series.min()
    return (series - min) / (max - min)


def read_data():
    player_df = pd.read_csv('Data/Season 13/Season13.csv')
    team_df = pd.read_csv('Data/Season 13/Season13TeamStats.csv')
    combine_df = pd.read_csv('Data/Season 13/Combine/season13-combine.csv')

    data_df = player_df.merge(team_df, on='team', how='outer')
    # data_df['skill'] = data_df['rating'] * data_df['tier'] * (data_df['w'] / data_df['mp'])

    data_df = data_df.dropna()
    data_df = data_df[data_df['gameCount'] > 3]
    data_df['scaledWinRate'] = feature_scale(data_df['w'] / data_df['mp'])
    data_df['scaledRating'] = feature_scale(data_df['rating'])
    data_df['skill'] = data_df['scaledWinRate'] + data_df['scaledRating'] + data_df['tier']
    # data_df['skill'] = feature_scale(data_df['skill'])

    # data_df.to_csv('Data/Season 13/Season13Merged.csv')
    # data_df = data_df.drop(columns=['mp', 'w', 'l', 'rd', 'team', 'name', 'gameCount'])
    # data_df = data_df.drop(columns=['name'])

    combine_df = combine_df.dropna()
    combine_df = combine_df[combine_df['gameCount'] > 3]
    combine_df = combine_df[combine_df['name'].isin(data_df['name'].unique())]
    combine_df = combine_df.merge(data_df[['name', 'skill']], on='name', how='outer')
    combine_df = combine_df.dropna()
    combine_df = combine_df[~combine_df['name'].duplicated(keep=False)]
    combine_df = combine_df.drop(columns=['name', 'team', 'kills', 'deaths', 'assists'])

    splits = RandomSplitter(valid_pct=0.2)(range_of(combine_df))
    dls = TabularPandas(combine_df, procs=[Categorify, FillMissing, Normalize], splits=splits,
                        y_names=['skill'], cat_names=['tier'], cont_names=
                        ['gameCount', 'rating', 'ctRating', 'TRating', 'kr', 'adr',
                         'kast', 'odr', 'impact', 'adp', 'utilDmg', 'ef', 'fAssists', 'util',
                         'hs', 'awpR', 'clutchR', 'suppR', 'suppXR', 'odaR', 'tradesR', 'tRatio',
                         'savesR', 'saveRate', 'cl_1', 'cl_2', 'cl_3', 'cl_4', 'cl_5', 'rounds',
                         'peak', 'pit', 'form', 'consistency', 'fiveK', 'fourK', 'multiR',
                         'threeK', 'twoK'])

    return dls.dataloaders()


def query_matches(seasons, url):
    query = '''
            {
              findManyMatch(where: {season: {in: ''' + str(seasons) + '''}}) {
                id
                matchDay
                mapName
                totalRounds
                tier
                season
                matchType
                rounds {
                  winnerClanName
                  winnerENUM
                  roundNum
                }
              }
            }
            '''
    headers = {
        'Content-Type': 'application/json',
    }

    response = requests.post(url, json={'query': query}, headers=headers)

    stats = response.json()['data']['findManyMatch']

    df = pd.DataFrame.from_dict(stats)

    df['rounds'] = df['rounds'].apply(lambda x: pd.DataFrame(x))
    scores = df['rounds'].apply(get_score)
    scores_df = pd.DataFrame(scores.to_list(), columns=['team1', 'score1', 'team2', 'score2'])
    df = df.join(scores_df)
    return df.drop(columns=['rounds'])


def query_player_stats(seasons, url):
    query = '''
    {
     findManyMatch(where: {season: {in: ''' + str(seasons) + '''}}) {
        matchStats {
          FAss
          wlp
          winPointsNormalizer
          winPoints
          utilThrown
          utilDmg
          twoK
          trades
          traded
          tr
          ticksAlive
          threeK
          teamsWinPoints
          teamClanName
          suppRounds
          suppDamage
          name
          smokeThrown
          side
          saves
          rws
          rwk
          rounds
          roundId
          rating
          ol
          ok
          nadesThrown
          nadeDmg
          mip
          matchId
          lurkerBlips
          lurkRounds
          kills
          killPoints
          killPointAvg
          kastRounds
          kast
          infernoDmg
          impactRating
          impactPoints
          iiwr
          hs
          health
          fourK
          flashThrown
          fiveK
          firesThrown
          entries
          enemyFlashTime
          ef
          eac
          drDiff
          deaths
          deathPlacement
          damageTaken
          damage
          ctWinPointsNormalizer
          ctWinPoints
          ctTeamsWinPoints
          ctRounds
          ctRating
          ctOL
          ctOK
          ctKills
          ctKASTRounds
          ctImpactRating
          ctKAST
          ctImpactPoints
          ctDeaths
          ctDamage
          ctAWP
          ctADR
          ctADP
          cl_5
          cl_4
          cl_3
          cl_2
          cl_1
          awpKills
          atd
          assists
          adr
          TWinPointsNormalizer
          TWinPoints
          TTeamsWinPoints
          TRounds
          TRating
          TRF
          TOL
          TOK
          TKills
          TKASTRounds
          TKAST
          TImpactRating
          TImpactPoints
          TDeaths
          TDamage
          TADR
          TADP
          RF
          RA
          KR
          steamID
        }
      }
    }
    '''
    headers = {
        'Content-Type': 'application/json',
    }
    response = requests.post(url, json={'query': query}, headers=headers)
    print(response)
    stats = response.json()['data']['findManyMatch']

    stats = [x['matchStats'] for x in stats]
    stats = [y for x in stats for y in x]
    df = pd.DataFrame(stats)
    df = df[df['TRounds'] + df['ctRounds'] != 0]
    return df


def get_score(df):
    df = df.drop_duplicates()
    df = df.drop(columns=['roundNum'])
    score = pd.DataFrame(df['winnerClanName'].value_counts().reset_index())
    scores = score.values.tolist()
    try:
        return scores[0] + scores[1]  # un-nest list
    except IndexError:
        return scores[0] + [None, 0]


def get_rounds_won(df):
    if df['teamClanName'] == df['team1']:
        return df['score1']
    return df['score2']


def divide_by_rounds(df, column_name):
    for column in df.columns:
        if column != column_name:  # inelegant but I can't be bothered to make it better
            df[column] = df[column] / df[column_name]
    return df


def fetch_stats_from_core(query, path):
    try:
        stats = pd.read_pickle(path)
    except FileNotFoundError:
        stats = query()
        stats.to_pickle(path)
    return stats


def prepare_stats(stats):
    game_count = stats[['name', 'tier']].groupby(['name']).count()
    game_count = game_count.rename(columns={'tier': 'gameCount'})
    stats = stats.merge(game_count, on='name', how='left')
    stats = stats[stats['gameCount'] >= 3]
    stats['tier'] = stats['tier'].apply(lambda x: tier_dict[x])
    stats['roundsWon'] = stats.apply(lambda x: get_rounds_won(x), axis=1)
    return stats


def process_regulation(stats):
    stats = prepare_stats(stats)
    stats = stats[['name', 'roundsWon', 'totalRounds', 'rating', 'tier']]
    stats['roundWinPercent'] = stats['roundsWon'] / stats['totalRounds']
    stats = stats.groupby(['name']).mean()
    stats['skill'] = (feature_scale(stats['rating']) + feature_scale(stats['roundWinPercent'])
                      + stats['tier'])
    stats = stats['skill']
    return stats


def process_combines(stats):
    stats = prepare_stats(stats)

    stats = stats.drop(columns=['score1', 'score2', 'team1', 'team2', 'matchType', 'mapName', 'matchDay', 'id',
                                'lurkerBlips', 'matchId', 'roundId', 'teamClanName', 'side', 'health'])

    stats_extras = stats[['name', 'gameCount', 'season', 'tier']]
    stats_extras = stats_extras.drop_duplicates().set_index(['name'])
    stats = stats.drop(columns=['gameCount', 'season', 'tier'])

    stats = stats.groupby(['name']).sum()
    stats = stats.astype('double')

    stats_t = stats[
        ['TRounds', 'TADR', 'TDamage', 'TDeaths', 'TImpactPoints', 'TImpactRating', 'TKAST',
         'TKASTRounds', 'TKills', 'TOL', 'TRF', 'TRating', 'TADP', 'TTeamsWinPoints', 'TWinPoints',
         'TWinPointsNormalizer']]

    # keeping T rounds for overall stats
    stats = stats.drop(columns=['TADR', 'TDamage', 'TDeaths', 'TImpactPoints', 'TImpactRating', 'TKAST',
                                'TKASTRounds', 'TKills', 'TOL', 'TRF', 'TRating', 'TADP', 'TTeamsWinPoints',
                                'TWinPoints', 'TWinPointsNormalizer'])

    stats_t = divide_by_rounds(stats_t, 'TRounds')
    stats_t = stats_t.drop(columns=['TRounds'])

    stats_ct = stats[
        ['ctRounds', 'ctADR', 'ctDamage', 'ctDeaths', 'ctImpactPoints', 'ctImpactRating', 'ctKAST',
         'ctKASTRounds', 'ctKills', 'ctOL', 'ctRating', 'ctADP', 'ctTeamsWinPoints', 'ctWinPoints',
         'ctWinPointsNormalizer']]

    # keeping cT rounds for overall stats
    stats = stats.drop(columns=['ctADR', 'ctDamage', 'ctDeaths', 'ctImpactPoints', 'ctImpactRating', 'ctKAST',
                                'ctKASTRounds', 'ctKills', 'ctOL', 'ctRating', 'ctADP', 'ctTeamsWinPoints',
                                'ctWinPoints', 'ctWinPointsNormalizer', 'rounds'])

    stats_ct = divide_by_rounds(stats_ct, 'ctRounds')
    stats_ct = stats_ct.drop(columns=['ctRounds'])

    stats = divide_by_rounds(stats, 'totalRounds')
    return stats, stats_t, stats_ct, stats_extras


def prepare_season_stats(stats, season):
    stats_previouse = stats[stats['season'] == season - 1]
    stats = stats[stats['season'] == season]


    stats_results = stats[stats['matchType'] == 'Regulation']
    stats_results = process_regulation(stats_results)
    
    #get results of previous season
    stats_previouse = stats_previouse[stats_previouse['matchType'] == 'Regulation']
    stats_previouse = process_regulation(stats_previouse)
    stats_previouse = stats_previouse.rename('previousSkill')

    stats = stats[stats['matchType'] == 'Combine']
    stats, stats_t, stats_ct, stats_extras = process_combines(stats)

    stats = stats.join([stats_t, stats_ct, stats_extras, stats_results, stats_previouse]).dropna()
    # TODO deal with player that changed tier in combines
    stats = stats.reset_index().drop_duplicates(subset='name', keep=False)
    stats = stats.set_index('name')

    return stats


def prepare_current_season(stats):
    stats = stats[stats['matchType'] == 'Combine']
    stats, stats_t, stats_ct, stats_extras = process_combines(stats)

    stats = stats.join([stats_t, stats_ct, stats_extras]).dropna()
    # TODO deal with player that changed tier in combines
    stats = stats.reset_index().drop_duplicates(subset='name', keep=False)
    stats = stats.set_index('name')
    return stats


def add_predictions(stats, learn):
    dl = learn.dls.test_dl(stats)
    raw_test_preds = learn.get_preds(dl=dl)
    stats['predictions'] = raw_test_preds[0].numpy()
    return stats




Fetch Data from stats API

In [115]:
if reFetch:
    team_stats = query_matches(active_seasons, url=stats_url)
    team_stats.to_pickle('TeamStats.pkl')
else:
    team_stats = pd.read_pickle('TeamStats.pkl')


In [116]:
if reFetch:
    player_game_stats = query_player_stats([13, 14], url=stats_url)
    player_game_stats = pd.concat([player_game_stats ,query_player_stats([15, 16], url=stats_url)])
    player_game_stats = pd.concat([player_game_stats ,query_player_stats([17], url=stats_url)])
    print(player_game_stats.shape)
    player_game_stats.to_pickle('PlayerStats.pkl')
else:
    player_game_stats = pd.read_pickle('PlayerStats.pkl')


In [117]:
# if reFetch:
#     season12_team_stats = query_matches([12], url='http://localhost:4000/graphql')
#     season12_team_stats.to_pickle('Season12TeamStats.pkl')

#     season12_player_stats = query_player_stats([12], url='http://localhost:4000/graphql')
#     season12_player_stats.to_pickle('Season12PlayerStats.pkl')
# else:
# season12_team_stats = pd.read_pickle('Season12TeamStats.pkl')
# season12_player_stats = pd.read_pickle('Season12PlayerStats.pkl')

# player_game_stats = pd.concat([player_game_stats, season12_player_stats])
# team_stats = pd.concat([team_stats, season12_team_stats])


In [118]:
player_game_stats["matchId"] = player_game_stats["matchId"].astype(str)
team_stats["id"] = team_stats["id"].astype(str)

stats = player_game_stats.merge(
    team_stats, left_on="matchId", right_on="id", how="outer"
)
stats


,FAss,wlp,winPointsNormalizer,winPoints,utilThrown,utilDmg,twoK,trades,traded,tr,...,matchDay,mapName,totalRounds,tier,season,matchType,team1,score1,team2,score2
0,0,1.050,85,23.97225,41,67,1,1,0,0.000000,...,M06,de_mirage,17.0,Recruit,16.0,Regulation,Blunder Battalion,13.0,Mist,4.0
1,0,0.000,85,4.38600,10,0,1,3,1,0.066667,...,M06,de_mirage,17.0,Recruit,16.0,Regulation,Blunder Battalion,13.0,Mist,4.0
2,0,0.000,85,5.58600,25,127,2,2,2,0.125000,...,M06,de_mirage,17.0,Recruit,16.0,Regulation,Blunder Battalion,13.0,Mist,4.0
3,0,0.000,85,5.74000,27,42,3,3,4,0.285714,...,M06,de_mirage,17.0,Recruit,16.0,Regulation,Blunder Battalion,13.0,Mist,4.0
4,1,6.863,85,9.02900,24,10,2,0,4,0.307692,...,M06,de_mirage,17.0,Recruit,16.0,Regulation,Blunder Battalion,13.0,Mist,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60574,1,4.300,110,10.96450,33,1,2,3,3,0.214286,...,M06,de_nuke,22.0,Elite,16.0,Regulation,Hydras,13.0,Walling Wizards,9.0
60575,0,0.000,110,16.04700,22,180,1,3,3,0.157895,...,M06,de_nuke,22.0,Elite,16.0,Regulation,Hydras,13.0,Walling Wizards,9.0
60576,0,1.600,110,25.95200,31,30,6,1,3,0.187500,...,M06,de_nuke,22.0,Elite,16.0,Regulation,Hydras,13.0,Walling Wizards,9.0
60577,0,0.000,110,24.60500,20,39,1,3,3,0.176471,...,M06,de_nuke,22.0,Elite,16.0,Regulation,Hydras,13.0,Walling Wizards,9.0


In [119]:

stats_13 = prepare_season_stats(stats, 14)
validate_stats = prepare_season_stats(stats, 15)
stats_cur = prepare_season_stats(stats, 16)

stats_13.head()


,FAss,wlp,winPointsNormalizer,winPoints,utilThrown,utilDmg,twoK,trades,traded,tr,...,ctRating,ctADP,ctTeamsWinPoints,ctWinPoints,ctWinPointsNormalizer,gameCount,season,tier,skill,previousSkill
name,,,,,,,,,,,,,,,,,,,,,
-Sainted-,0.075000,0.000000,5.0,0.920831,2.029167,3.754167,0.150000,0.137500,0.137500,0.009007,...,0.087174,0.223212,3.113000,0.758157,5.0,11,14.0,3,3.721503,4.315348
.caM,0.000000,0.043667,5.0,0.925209,1.712121,2.560606,0.181818,0.136364,0.151515,0.009543,...,0.081531,0.224471,5.143618,0.817986,5.0,3,14.0,5,6.117024,6.016125
1000-Denials,0.039474,0.153868,5.0,0.950076,2.236842,5.644737,0.092105,0.118421,0.078947,0.007080,...,0.111598,0.291667,4.562703,0.653188,5.0,4,14.0,4,4.802101,4.070734
AJay,0.030769,0.019385,5.0,0.706262,1.707692,9.461538,0.046154,0.107692,0.123077,0.007457,...,0.078733,0.249656,3.579030,0.732879,5.0,3,14.0,5,6.031811,4.868967
ARealPigeon,0.025641,0.000000,5.0,0.812000,1.628205,7.230769,0.153846,0.141026,0.153846,0.007252,...,0.082598,0.225000,4.037960,0.967222,5.0,3,14.0,2,3.339130,2.837887


In [120]:
#season 13 csv has a different format
facit_mmr_13 = pd.read_csv('s13_train_set.csv').drop(columns=['id', 'pulls', 'metrics_err', 'recency_weighted_metrics_err'])

facit_mmr_14 = pd.read_csv('s14_train_set.csv').drop(columns=['id', 'pulls'])
facit_mmr_14['name'] = facit_mmr_14['name'].apply(lambda x: x.split("'")[1]) # remove quotes
facit_mmr_cur = pd.read_csv('s15_train_set.csv').drop(columns=['id', 'pulls'])
facit_mmr_cur['name'] = facit_mmr_cur['name'].apply(lambda x: x.split("'")[1]) # remove quotes


stats_13 = stats_13.join(facit_mmr_13.set_index('name'), rsuffix='_facit').reset_index().dropna()
validate_stats = validate_stats.join(facit_mmr_14.set_index('name'), rsuffix='_facit').reset_index().dropna()
stats_cur = stats_cur.join(facit_mmr_cur.set_index('name'), rsuffix='_facit').reset_index().dropna()

stats_cur.head()



,name,FAss,wlp,winPointsNormalizer,winPoints,utilThrown,utilDmg,twoK,trades,traded,...,tier,skill,previousSkill,avg_rating,avg_lobby_elo,adjusted_elo,recency_weighted_adjusted_elo,consistency,mmr,stats_mmr_err
1,Akuryo,0.021687,0.021957,4.987952,0.634842,1.190361,3.233735,0.089157,0.106024,0.125301,...,5,6.064577,6.225529,0.972824,1801.604706,1781.877661,1823.286768,0.406699,499.330493,43.432591
4,Amp3D,0.030717,0.033703,5.000000,0.881907,1.870307,4.645051,0.122867,0.081911,0.122867,...,1,2.813591,2.144248,0.933635,1127.007527,1070.168226,1082.603727,0.453659,226.263804,33.858155
5,Arcane,0.000000,0.000000,5.000000,0.939683,1.433333,4.350000,0.166667,0.100000,0.166667,...,5,6.023609,5.883698,1.107558,1603.311842,1659.871474,1689.214910,0.237500,596.327383,35.462562
6,Arch3r,0.024194,0.040702,5.000000,1.234480,1.056452,2.225806,0.112903,0.161290,0.088710,...,4,6.193496,4.885602,1.221568,1376.928049,1515.641058,1556.015977,0.891304,385.399211,20.275033
8,Ayzore,0.020134,0.076832,4.983221,0.998651,1.795302,6.614094,0.157718,0.127517,0.120805,...,1,2.121924,2.159095,0.860397,807.827451,739.012339,744.076220,0.113839,265.957274,35.510732


In [121]:

stats = pd.concat([stats_13, validate_stats], ignore_index=True)
# stats = validate_stats

print(stats.shape)


(436, 111)


In [122]:
cat_names = [] #['season']
cont_names = list(filter(lambda x: x not in cat_names + ['skill', 'steamID', 'name'], stats.columns))

splits = RandomSplitter(valid_pct=0.2)(range_of(stats))
to = TabularPandas(stats, procs=[Categorify, FillMissing, Normalize], splits=splits, y_names=['skill'],
                    cat_names=cat_names, cont_names=cont_names, device=torch.device('cuda'))

dls = to.dataloaders(bs=64)


In [123]:
print(to.show(5))


,FAss,wlp,winPointsNormalizer,winPoints,utilThrown,utilDmg,twoK,trades,traded,tr,ticksAlive,threeK,teamsWinPoints,suppRounds,suppDamage,smokeThrown,saves,rws,rwk,rating,ol,ok,nadesThrown,nadeDmg,mip,lurkRounds,kills,killPoints,killPointAvg,kastRounds,kast,infernoDmg,impactRating,impactPoints,iiwr,hs,fourK,flashThrown,fiveK,firesThrown,entries,enemyFlashTime,ef,eac,drDiff,deaths,deathPlacement,damageTaken,damage,ctRounds,ctOK,ctAWP,cl_5,cl_4,cl_3,cl_2,cl_1,awpKills,atd,assists,adr,TRounds,TOK,RF,RA,KR,totalRounds,roundsWon,TADR,TDamage,TDeaths,TImpactPoints,TImpactRating,TKAST,TKASTRounds,TKills,TOL,TRF,TRating,TADP,TTeamsWinPoints,TWinPoints,TWinPointsNormalizer,ctADR,ctDamage,ctDeaths,ctImpactPoints,ctImpactRating,ctKAST,ctKASTRounds,ctKills,ctOL,ctRating,ctADP,ctTeamsWinPoints,ctWinPoints,ctWinPointsNormalizer,gameCount,season,tier,previousSkill,avg_rating,avg_lobby_elo,adjusted_elo,recency_weighted_adjusted_elo,consistency,mmr,stats_mmr_err,skill
117,0.045455,0.024448,5.136364,0.799531,2.094406,3.482517,0.111888,0.108392,0.083916,0.005232,1893.891602,0.038462,4.357103,0.202797,10.804195,0.625874,3.146853e-02,0.334208,0.493007,0.039578,0.066434,0.101399,0.290210,1.790210,0.083916,0.031469,0.737762,0.868088,0.045385,0.741259,0.028943,1.685315,0.038637,1.293809,0.023463,0.307692,0.013986,0.706294,3.496503e-03,0.472028,0.048951,1.494809,0.465035,0.297203,0.030100,0.646853,0.100292,72.181816,73.125877,0.517483,0.052448,0.052448,0.0,0.000000,0.0,0.000000,0.013986,0.059441,1.094406,0.202797,2.784858,0.482517,0.048951,0.517483,0.482517,0.028262,286.0,0.517483,6.388210,76.891304,0.688406,1.429685,0.087881,0.058899,0.702899,0.782609,0.086957,0.485507,0.091032,0.208319,4.237208,0.827866,5.115942,4.981118,69.614868,0.608108,1.167113,0.067986,0.057462,0.777027,0.695946,0.047297,0.070814,0.191968,4.468897,0.773111,5.155406,11.0,14.0,3.0,3.914235,0.863068,1422.060059,1326.587769,1329.892456,1.818182,426.163666,25.125751,4.275760
171,0.017391,0.045061,5.000000,0.786364,1.078261,2.634783,0.069565,0.104348,0.078261,0.004896,2169.356445,0.043478,3.952101,0.156522,9.808696,0.295652,2.608696e-02,0.387916,0.452174,0.038569,0.052174,0.113043,0.095652,1.278261,0.069565,0.034783,0.634783,0.736289,0.050604,0.652174,0.027906,1.321739,0.039125,1.105698,0.029715,0.391304,0.008696,0.365217,1.084202e-19,0.321739,0.026087,0.608285,0.200000,0.217391,-0.478544,0.686957,0.126640,74.695656,66.591301,0.513043,0.069565,0.034783,0.0,0.000000,0.0,0.008696,0.017391,0.043478,1.460870,0.139130,2.835466,0.486957,0.043478,0.478261,0.521739,0.026796,115.0,0.478261,6.160714,73.375000,0.625000,1.330165,0.082822,0.062202,0.714286,0.767857,0.053571,0.642857,0.082292,0.248845,5.355416,1.148841,5.000000,5.099718,60.152542,0.745763,0.892644,0.072858,0.051130,0.593220,0.508475,0.050847,0.072131,0.260878,2.620140,0.442318,5.000000,5.0,14.0,3.0,3.390116,1.031561,1235.719971,1272.520264,1326.600830,0.476190,391.867798,59.811253,4.251917
329,0.031546,0.009864,5.000000,0.644145,1.637224,1.299685,0.097792,0.088328,0.123028,0.007658,2219.110352,0.034700,3.842500,0.179811,10.883281,0.523659,1.261830e-02,0.332790,0.391167,0.038594,0.113565,0.104101,0.034700,0.000000,0.066246,0.018927,0.567823,0.694973,0.057687,0.627760,0.029444,1.293375,0.039066,1.037528,0.027542,0.233438,0.003155,0.772871,1.084202e-19,0.305994,0.063091,1.948610,0.583596,0.315457,-1.132741,0.772871,0.127026,84.186119,61.230286,0.488959,0.041009,0.107256,0.0,0.000000,0.0,0.006309,0.003155,0.242902,1.615142,0.186120,2.852732,0.511041,0.063091,0.457413,0.542587,0.026476,317.0,0.457413,5.676020,62.679012,0.802469,1.122824,0.076447,0.059746,0.641975,0.592593,0.098765,0.469136,0.078316,0.255583,3.998313,0.691219,5.000000,5.603584,59.716129,0.741935,0.948380,0.076475,0.058781,0.612903,0.541936,0.129032,0.075724,0.237199,3.679650,0.594946,5.000000,15.0,15.0,3.0,3.885634,0.951984,1263.423096,1215.851074,1280.128784,0.406250,429.709320,23.850830,3.893589
263,0.065789,0.096645,5.000000,1.199364,1.828947,4.256579,0.151316,0.111842

None


In [124]:
import optuna

def objective(trial, dls):
    # Suggest hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    wd = trial.suggest_float('wd', 1e-6, 1e-2, log=True)
    layers_depth = trial.suggest_int('layers_depth', 100, 1000)
    layers_width = trial.suggest_int('layers_width', 100, 1000)

    y_range = torch.tensor([0, 8])
    tc = tabular_config(ps=[0.001, 0.01], embed_p=0.04, y_range=y_range)

    # Create a new learner with suggested hyperparameters
    learn = tabular_learner(dls, wd=wd, cbs=[EarlyStoppingCallback(patience=30)],
                            loss_func=MSELossFlat(), config=tc,
                            layers=[layers_depth, layers_width])
    learn.model.to('cuda')

    # Train the model with early stopping
    with learn.no_logging(), learn.no_bar():
        learn.fit(30, lr)

    # Return the validation accuracy
    # test = learn.recorder.log
    return learn.recorder.log[2]


# Create an Optuna study and optimize the objective function

def tune_hyperparams(dls):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, dls), n_trials=100, show_progress_bar=True)

    return study.best_trial


c:\Users\Shade\Documents\Eric\CSC_bull_Shiza\Tech_Committee\CSC-PP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [125]:
if reTrainHyper:
    hyper_params = tune_hyperparams(dls).params
    print(hyper_params)


In [126]:
if reTrain:
    y_range = torch.tensor([0, 8])
    tc = tabular_config(ps=[0.001, 0.01], embed_p=0.04, y_range=y_range)
    learn = tabular_learner(dls, wd=hyper_params['wd'], cbs=[EarlyStoppingCallback(patience=30), SaveModelCallback()],
                                loss_func=MSELossFlat(), config=tc,
                                layers=[hyper_params['layers_depth'], hyper_params['layers_width']])
    learn.fit(300, lr=hyper_params['lr'])
    
    learn.export('myModel')


In [127]:
learn = load_learner('myModel')

# validate_stats['season'] = 13
# stats_cur['season'] = 13
# stats = add_predictions(stats, learn).reset_index()[['name', 'gameCount', 'tier', 'skill', 'predictions']]
# validate_stats = add_predictions(validate_stats, learn).reset_index()[
#     ['name', 'gameCount', 'tier', 'skill', 'predictions']]
season15prediction = add_predictions(stats_cur, learn).reset_index()[['name', 'gameCount', 'tier', 'skill', 'predictions']]


In [128]:

playerMMr = pd.read_csv('backup_mmrs_s15.csv')
playerMMr = playerMMr[['name', 'mmr']]
season15prediction = season15prediction.merge(playerMMr, on='name', how='left').dropna()


In [129]:

# stats.to_csv("season13_stats.csv")
# validate_stats.to_csv("season14_stats.csv")
# stats_cur.to_csv("season15_predictions.csv")


In [130]:
season15prediction['predicted_rank'] = season15prediction['predictions'].rank(ascending=False)
season15prediction['mmr_rank'] = season15prediction['mmr'].rank(ascending=False)
season15prediction['actual_rank'] = season15prediction['skill'].rank(ascending=False)

season15prediction['prediction_diff'] = abs(season15prediction['predicted_rank'] - season15prediction['actual_rank'])
season15prediction['mmr_diff'] = abs(season15prediction['mmr_rank'] - season15prediction['actual_rank'])

print(f'MMR ranking error: {season15prediction["mmr_diff"].mean()} median: {season15prediction["mmr_diff"].median()} std: {season15prediction["mmr_diff"].std()}')
print(f'Predicted ranking error: {season15prediction["prediction_diff"].mean()} median: {season15prediction["prediction_diff"].median()}  std: {season15prediction["prediction_diff"].std()}')


MMR ranking error: 13.184615384615384 median: 12.0 std: 10.393822416443706
Predicted ranking error: 11.723076923076922 median: 9.5  std: 9.905769507156963


In [131]:
season15prediction


,name,gameCount,tier,skill,predictions,mmr,predicted_rank,mmr_rank,actual_rank,prediction_diff,mmr_diff
0,Akuryo,18,5,6.064577,5.481649,643.0,11.0,9.0,14.0,3.0,5.0
1,Amp3D,14,1,2.813591,2.113994,180.0,121.0,126.0,106.0,15.0,20.0
2,Arcane,3,5,6.023609,6.011236,678.0,7.0,7.0,17.0,10.0,10.0
3,Arch3r,6,4,6.193496,4.811990,492.0,29.0,31.5,9.0,20.0,22.5
4,Ayzore,13,1,2.121924,2.455353,262.0,112.0,116.0,122.0,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
125,steez,16,2,3.381444,3.112158,349.0,91.0,80.0,93.0,2.0,13.0
126,stranger,9,4,5.914604,5.399673,578.0,13.0,16.0,23.0,10.0,7.0
127,tripleman,9,1,1.547404,1.709981,124.0,130.0,129.0,130.0,0.0,1.0
128,ukcats,7,1,2.682105,2.202986,275.0,119.0,109.0,111.0,8.0,2.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82e613b0-1362-495d-a2df-baa1d856f8aa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>